# Stripe - Capital Lending Performance for Small Business Revenue

```SQL
CREATE TABLE fct_loans (
    loan_id integer,
    business_id integer,
    loan_amount decimal,
    loan_issued_date date,
    loan_repaid boolean
);

CREATE TABLE dim_businesses(
    business_id integer,
    monthly_revenue decimal,
    revenue_variability decimal,
    business_size varchar
);

INSERT INTO fct_loans (loan_id, business_id, loan_amount, loan_repaid, loan_issued_date)
VALUES
    (1, 1, 10000.00, TRUE, '2024-01-05'),
    (2, 3, 5000.00, FALSE, '2024-01-08'),
    (3, 4, 20000.00, TRUE, '2024-01-15'),
    (4, 5, 15000.00, TRUE, '2024-01-20'),
    (5, 7, 8000.00, FALSE, '2024-01-25'),
    (6, 2, 12000.00, TRUE, '2024-01-12'),
    (7, 2, 9000.00, TRUE, '2024-01-18'),
    (8, 8, 11000.00, FALSE, '2024-01-22'),
    (9, 9, 7000.00, TRUE, '2024-01-28'),
    (10, 10, 13000.00, FALSE, '2024-01-30'),
    (11, 3, 6000.00, FALSE, '2024-01-14'),
    (12, 4, 5000.00, TRUE, '2024-01-09'),
    (13, 7, 9000.00, TRUE, '2024-01-27'),
    (14, 8, 10000.00, FALSE, '2024-01-16'),
    (15, 6, 14000.00, TRUE, '2024-01-11'),
    (16, 1, 25000.00, TRUE, '2024-01-07');

INSERT INTO dim_businesses (business_id, monthly_revenue, revenue_variability, business_size)
VALUES
    (1, 10000.00, 0.05, 'small'),
    (2, 15000.00, 0.08, 'small'),
    (3, 8000.00, 0.25, 'small'),
    (4, 20000.00, 0.30, 'small'),
    (5, 12000.00, 0.22, 'small'),
    (6, 9000.00, 0.07, 'small'),
    (7, 11000.00, 0.55, 'small'),
    (8, 13000.00, 0.40, 'small'),
    (9, 14000.00, 0.06, 'small'),
    (10, 16000.00, 0.50, 'small'),
    (11, 30000.00, 0.10, 'medium'),
    (12, 50000.00, 0.12, 'large'),
    (13, 12500.00, 0.15, 'small');


SELECT * FROM  fct_loans;

SELECT * FROm dim_businesses;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_business = pd.read_csv('Data/023/dim_businesses.csv')
df_loans = pd.read_csv('Data/023/fct_loans.csv',parse_dates=['loan_issued_date'])

df_business.head()

,business_id,monthly_revenue,revenue_variability,business_size
0,1,10000,0.05,small
1,2,15000,0.08,small
2,3,8000,0.25,small
3,4,20000,0.30,small
4,5,12000,0.22,small


In [3]:
df_loans.head()

,loan_id,business_id,loan_amount,loan_issued_date,loan_repaid
0,1,1,10000,2024-01-05,True
1,2,3,5000,2024-01-08,False
2,3,4,20000,2024-01-15,True
3,4,5,15000,2024-01-20,True
4,5,7,8000,2024-01-25,False


# Pregunta 1

### ¿Cuál es el ingreso mensual promedio de las empresas pequeñas que recibieron un préstamo frente a las que no lo recibieron durante enero de 2024? Utilice el campo 'business_size' para filtrar por empresas pequeñas (small).

In [8]:
# 1. Unir manteniendo todos los negocios
df_merge = df_business.merge(df_loans, on='business_id', how='left')

# 2. Filtrar solo los Small
df_small = df_merge[df_merge['business_size'] == 'small'].copy()

# 3. Crear los grupos (si loan_id es NaN, no tuvo préstamo)
df_small['group'] = df_small['loan_id'].isna().map({True: 'No Loan', False: 'Received Loan'})

# 4. Calcular el promedio
resultado = df_small.groupby('group')['monthly_revenue'].mean().reset_index()
resultado

,group,monthly_revenue
0,No Loan,12500.0
1,Received Loan,12812.5


```SQL
SELECT 
    CASE 
        WHEN l.loan_id IS NULL THEN 'No Loan' 
        ELSE 'Received Loan' 
    END AS loan_group,
    AVG(b.monthly_revenue) AS avg_revenue
FROM dim_businesses b
LEFT JOIN fct_loans l 
    ON b.business_id = l.business_id 
    AND l.loan_issued_date BETWEEN '2024-01-01' AND '2024-01-31'
WHERE b.business_size = 'small'
GROUP BY 1;
```

# Pregunta 2


#### Para los préstamos otorgados a empresas pequeñas en enero de 2024, ¿qué porcentaje se reembolsó con éxito? Clasifique estos préstamos según la variabilidad de los ingresos de la empresa prestataria (baja, media o alta) utilizando estos valores:

- Baja: < 0.1

- Media: 0.1 - 0.3 (inclusive)

- Alta: > 0.3"

In [17]:
# 1. Filtro y Merge (Correcto)
df_small = df_business[df_business['business_size'] == 'small'].merge(df_loans, on='business_id')
df_small = df_small[df_small['loan_issued_date'].between('2024-01-01','2024-01-31')]

# 2. Corregimos los límites (bins)
# Low: < 0.1 | Medium: 0.1 a 0.3 | High: > 0.3
bins = [-float('inf'), 0.1, 0.3, float('inf')]
labels = ['Low', 'Medium', 'High']

# Usamos include_lowest=True para asegurar que el 0 entre en 'Low'
df_small['variability_tier'] = pd.cut(df_small['revenue_variability'], bins=bins, labels=labels, include_lowest=True)

# 3. Cálculo de la tasa de pago (Correcto, arreglando el typo en 'analysis')
repayment_analysis = df_small.groupby('variability_tier', observed=False)['loan_repaid'].mean() * 100

repayment_analysis = repayment_analysis.reset_index()

repayment_analysis

,variability_tier,loan_repaid
0,Low,100.0
1,Medium,60.0
2,High,20.0


```SQL
SELECT
    CASE
        WHEN b.revenue_variability < 0.1 THEN 'Low'
        WHEN b.revenue_variability <= 0.3 THEN 'Medium'
        ELSE 'High'
    END AS variability_tier,
    COUNT(CASE WHEN l.loan_repaid = TRUE THEN 1 END) * 100.0 / COUNT(*) AS repayment_rate
FROM fct_loans l
JOIN dim_businesses b ON l.business_id = b.business_id
WHERE b.business_size = 'small'
    AND l.loan_issued_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY 1
ORDER BY repayment_rate DESC;
```

# Pregunta 3

### Para las empresas pequeñas durante enero de 2024, ¿cuál es la tasa de éxito en el pago de préstamos para cada categoría de variabilidad de ingresos? Ordene los resultados de mayor a menor tasa de éxito para evaluar la correlación entre la variabilidad de los ingresos y la confiabilidad en el pago.

In [18]:
resultado = (df_small.groupby('variability_tier', observed=False)['loan_repaid'].mean() * 100)

resultado = resultado.reset_index(name='success_rate').sort_values(by='success_rate', ascending=False)

resultado

,variability_tier,success_rate
0,Low,100.0
1,Medium,60.0
2,High,20.0


```SQL
SELECT
    CASE
        WHEN b.revenue_variability < 0.1 THEN 'Low'
        WHEN b.revenue_variability <= 0.3 THEN 'Medium'
        ELSE 'High'
    END AS variagility_category,
    AVG(CASE WHEN l.loan_repaid = TRUE THEN 1.0 ELSE 0.0 END) * 100.0 AS success_rate
FROM fct_loans l
JOIN dim_businesses b ON l.business_id = b.business_id
WHERE b.business_size = 'small'
    AND l.loan_issued_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY 1
ORDER BY success_rate DESC;
```